In [19]:
import spacy
import pickle
import random
import os
for dirname, _, filenames in os.walk('sample_data/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sample_data/data/Smith Resume.docx
sample_data/data/lexemes.bin
sample_data/data/key2row
sample_data/data/vectors
sample_data/data/tokenizer
sample_data/data/Alice Clark CV.docx
sample_data/data/train_data.pkl
sample_data/data/Alice Clark CV.txt
sample_data/data/cfg
sample_data/data/Smith Resume.pdf
sample_data/data/meta.json
sample_data/data/model
sample_data/data/base_config.cfg
sample_data/data/strings.json
sample_data/data/lookups.bin
sample_data/data/Alice Clark CV.pdf
sample_data/data/README.md
sample_data/data/moves
sample_data/data/train_data.json
sample_data/data/abir ben brahem.pdf


In [20]:
train_data = pickle.load(open('sample_data/train_data.pkl','rb'))

In [21]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [22]:
import spacy

nlp = spacy.load('en_core_web_sm')


In [23]:
!pip install spacy-lookups-data


In [25]:
nlp=spacy.blank("en")
from spacy.training.example import Example

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner')
    else:
        ner = nlp.get_pipe('ner')

    for _, annotations in train_data:
        for ent in annotations.get('entities', []):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                try:
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    nlp.update(
                        [example],  # batch of Examples
                        drop=0.2,  # dropout - make it harder to memorize data
                        losses=losses)
                except Exception as e:
                    pass

            print(losses)

# Let's train the model



In [26]:
train_model(train_data)

Starting iteration 0
{'ner': 7881.7387181310005}
Starting iteration 1
{'ner': 2616.610502436979}
Starting iteration 2
{'ner': 2931.469126785822}
Starting iteration 3
{'ner': 2733.8846687079854}
Starting iteration 4
{'ner': 2330.084295155717}
Starting iteration 5
{'ner': 1783.6385194691939}
Starting iteration 6
{'ner': 1581.7218033977656}
Starting iteration 7
{'ner': 1421.5854402642142}
Starting iteration 8
{'ner': 1242.6828361721987}
Starting iteration 9
{'ner': 1205.2297867163422}


In [27]:
nlp.to_disk('nlp_model')

In [28]:
!pip install PyMuPDF

In [33]:
import sys
import fitz

fname = 'sample_data/data/Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())

tx = " ".join(text.split('\n'))  # for removing the next line character '/n'
print(tx)


Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [34]:
# Now we will pass this extracted text to our model
nlp_model = nlp.from_disk('nlp_model/')

doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark  AI
LOCATION                      - Delhi
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
LOCATION                      - Store
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - Indian Institute of Technology – Mumbai
GRADUATION YEAR               - 2001
